In [ ]:
from Bio import SeqIO
from tqdm.auto import tqdm
tqdm.pandas()

## After creation of CDS file with Bedtools

### Parsing new CDS file

In [ ]:
cds_file = "Ahypochondriacus_315_v1.0.gene.cter_trimmed.from_bed.fa"
with open(cds_file, 'r') as fin:
    cds_records = list(SeqIO.parse(fin, format='fasta'))
print(f"Number of CDS sequences: {len(cds_records)}")
cds_ids = [rec.id for rec in cds_records]
cds_record_lengths = [len(rec.seq) for rec in cds_records]
length_sum = sum(cds_record_lengths)
print(f"Cumulated length of all CDS: {length_sum}")

### Checking if the new sequences are subset of the original ones

In [ ]:
original_cds_file = "Phytozome/PhytozomeV12/Ahypochondriacus/annotation/Ahypochondriacus_315_v1.0.cds_primaryTranscriptOnly.fa"

In [ ]:
with open(original_cds_file, 'r') as fin:
    original_cds_records = list(SeqIO.parse(fin, format='fasta'))
original_cds_records = sorted(original_cds_records, key=lambda rec: rec.id)
original_cds_ids = [rec.id for rec in original_cds_records]
original_cds_record_lengths = [len(rec.seq) for rec in original_cds_records]
original_length_sum = sum(original_cds_record_lengths)

In [ ]:
# checking that all cds sequences are subset of the original ones
# it's not a problem if new sequences are present, they were in the annotation file anyway
for cds_id in cds_ids:
    if not cds_id.split(".")[0] in original_cds_ids:
        print(f"Record {cds_id} not found in original file")
    

### Checking if the size ratio is respected

In [ ]:
target_length_sum = 2E7
trimming_ratio = 1 - target_length_sum / original_length_sum
trimming_ratio

In [ ]:
tolerance = 0.1
expected_length_ratio = 1 - trimming_ratio
min_ratio = expected_length_ratio * (1 - tolerance)
max_ratio = expected_length_ratio * (1 + tolerance)

for record in tqdm(cds_records):
    for original_record in original_cds_records:
        # checking if the new sequence is contained in the original one
        if record.id.split(".")[0] == original_record.id:
            
            if not record.seq in original_record.seq:
                raise ValueError(f'Sequence of {record.id} is not included in the original one')
            
            # checking if the size ratio is respected
            actual_ratio = len(record.seq) / len(original_record.seq)
            if not min_ratio <= actual_ratio <= max_ratio:
                raise ValueError(f'Ratio of {record.id}: {actual_ratio}  is not between {min_ratio} and {max_ratio}')
        